# Section 05: Data Cleaning in Pandas

1. Map and Apply
2. Groupby
3. Joining DataFrames
4. Missing Values

## `.map()` and `.apply()`

These are two methods used to run functions across multiple rows/columns of your DataFrame, and there are multiple ways to use each.

In [2]:
import numpy as np
import pandas as pd

In [3]:
car_df = pd.read_csv('http://faculty.marshall.usc.edu/gareth-james/ISL/Auto.csv', na_values='?')

In [4]:
car_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [5]:
car_df.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
count,397.000000,397.000000,397.000000,392.000000,397.000000,397.000000,397.000000,397.000000
mean,23.515869,5.458438,193.532746,104.469388,2970.261965,15.555668,75.994962,1.574307
std,7.825804,1.701577,104.379583,38.491160,847.904119,2.749995,3.690005,0.802549
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.000000,75.000000,2223.000000,13.800000,73.000000,1.000000
50%,23.000000,4.000000,146.000000,93.500000,2800.000000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,126.000000,3609.000000,17.100000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


### a) Broadcasting simple operations

i.e. convert `year` into its full year by adding `1900` to each value (we're assuming that the year range of 70-82 is in the 1900s)

In [6]:
car_df['year'] + 1900 # not destructive, we'd have to reassign the new values

0      1970
1      1970
2      1970
3      1970
4      1970
       ... 
392    1982
393    1982
394    1982
395    1982
396    1982
Name: year, Length: 397, dtype: int64

In [7]:
car_df['year'] = car_df['year'] + 1900 # now we're changing it

In [8]:
car_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,1970,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,1970,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,1970,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,1970,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,1970,1,ford torino


### b) Boolean masking

(also useful to understand this for filtering dataframes)

In [9]:
car_df[['mpg', 'cylinders', 'displacement']]  % 2 == 0

,mpg,cylinders,displacement
0,True,True,False
1,False,True,True
2,True,True,True
3,True,True,True
4,False,True,True
...,...,...,...
392,False,True,True
393,True,True,False
394,True,True,False
395,True,True,True


In [10]:
car_df['weight'] > 2800 # outputs a series of booleans
# showing whether each value meets the condition set

0       True
1       True
2       True
3       True
4       True
       ...  
392    False
393    False
394    False
395    False
396    False
Name: weight, Length: 397, dtype: bool

We can use this to create new, perhaps more useful columns!

In [11]:
car_df['heavy'] = car_df['weight'] > 2800

In [12]:
car_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,heavy
0,18.0,8,307.0,130.0,3504,12.0,1970,1,chevrolet chevelle malibu,True
1,15.0,8,350.0,165.0,3693,11.5,1970,1,buick skylark 320,True
2,18.0,8,318.0,150.0,3436,11.0,1970,1,plymouth satellite,True
3,16.0,8,304.0,150.0,3433,12.0,1970,1,amc rebel sst,True
4,17.0,8,302.0,140.0,3449,10.5,1970,1,ford torino,True


### c. The `.map()` function

`.map()` only works on Series, or on single columns!

You can supply a condition directly into the lambda, or define specific functions.

In [13]:
car_df['year'].map(lambda y: y < 80)

0      False
1      False
2      False
3      False
4      False
       ...  
392    False
393    False
394    False
395    False
396    False
Name: year, Length: 397, dtype: bool

In [14]:
# tangent - for car names - let's see what names exist
# unique(), nunique(), value_counts()
car_df['name']

0      chevrolet chevelle malibu
1              buick skylark 320
2             plymouth satellite
3                  amc rebel sst
4                    ford torino
                 ...            
392              ford mustang gl
393                    vw pickup
394                dodge rampage
395                  ford ranger
396                   chevy s-10
Name: name, Length: 397, dtype: object

In [15]:
car_df['name'].unique()

array(['chevrolet chevelle malibu', 'buick skylark 320',
       'plymouth satellite', 'amc rebel sst', 'ford torino',
       'ford galaxie 500', 'chevrolet impala', 'plymouth fury iii',
       'pontiac catalina', 'amc ambassador dpl', 'dodge challenger se',
       "plymouth 'cuda 340", 'chevrolet monte carlo',
       'buick estate wagon (sw)', 'toyota corona mark ii',
       'plymouth duster', 'amc hornet', 'ford maverick', 'datsun pl510',
       'volkswagen 1131 deluxe sedan', 'peugeot 504', 'audi 100 ls',
       'saab 99e', 'bmw 2002', 'amc gremlin', 'ford f250', 'chevy c20',
       'dodge d200', 'hi 1200d', 'chevrolet vega 2300', 'toyota corona',
       'ford pinto', 'plymouth satellite custom', 'ford torino 500',
       'amc matador', 'pontiac catalina brougham', 'dodge monaco (sw)',
       'ford country squire (sw)', 'pontiac safari (sw)',
       'amc hornet sportabout (sw)', 'chevrolet vega (sw)',
       'pontiac firebird', 'ford mustang', 'mercury capri 2000',
       'opel 1900'

In [16]:
car_df['name'].map(lambda n: 'ford' in n)

0      False
1      False
2      False
3      False
4       True
       ...  
392     True
393    False
394    False
395     True
396    False
Name: name, Length: 397, dtype: bool

In [17]:
# for non-Boolean outputs, you can define a function to feed into the lambda

def car_brand(value):
    if 'ford' in value:
        return 'Ford'
    elif 'buick' in value:
        return 'Buick'
    elif 'chevrolet' in value:
        return 'Chevrolet'
    else:
        return 'Other'
    

In [18]:
car_df['name'].map(lambda n: car_brand(n))

0      Chevrolet
1          Buick
2          Other
3          Other
4           Ford
         ...    
392         Ford
393        Other
394        Other
395         Ford
396        Other
Name: name, Length: 397, dtype: object

In [19]:
# the above in not destructive. To assign it: 
car_df['brand'] = car_df['name'].map(lambda n: car_brand(n))

In [20]:
car_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,heavy,brand
0,18.0,8,307.0,130.0,3504,12.0,1970,1,chevrolet chevelle malibu,True,Chevrolet
1,15.0,8,350.0,165.0,3693,11.5,1970,1,buick skylark 320,True,Buick
2,18.0,8,318.0,150.0,3436,11.0,1970,1,plymouth satellite,True,Other
3,16.0,8,304.0,150.0,3433,12.0,1970,1,amc rebel sst,True,Other
4,17.0,8,302.0,140.0,3449,10.5,1970,1,ford torino,True,Ford


### d. The `.apply()` function

This works on a DataFrame, or on multiple columns of your DataFrame.


Before that, `applymap()`, **a different method**, is like `.map()` but you can do functions across multiple columns or on an entire DataFrame. 


i.e. turning all your data into strings

In [21]:
car_df.applymap(lambda value: str(value))['displacement']

# remember this isn't destructive until you reassign it to the variable!

0      307.0
1      350.0
2      318.0
3      304.0
4      302.0
       ...  
392    140.0
393     97.0
394    135.0
395    120.0
396    119.0
Name: displacement, Length: 397, dtype: object

In [22]:
new = car_df.applymap(lambda value: str(value))['displacement']

In [23]:
car_df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name', 'heavy', 'brand'],
      dtype='object')

`.apply()` allows us to perform specific functions that take different columns as inputs. For example, if we want to engineer a new variable that's some combination of other variables. I usually define the function I want to use in `.apply()` taking in a row as its argument.


Let's create a variable that finds the horsepower per weight of the car.

In [24]:
def horsepower_per_weight(row):
    return row['horsepower']/row['weight']

In [25]:
car_df.apply(lambda r: horsepower_per_weight(r), axis=1) # axis = 1 is for columns

0      0.037100
1      0.044679
2      0.043655
3      0.043694
4      0.040591
         ...   
392    0.030824
393    0.024413
394    0.036601
395    0.030095
396    0.030147
Length: 397, dtype: float64

In [26]:
# make the column
car_df['hp_per_weight'] = car_df.apply(lambda r: horsepower_per_weight(r), axis=1) # axis = 1 is for columns
car_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,heavy,brand,hp_per_weight
0,18.0,8,307.0,130.0,3504,12.0,1970,1,chevrolet chevelle malibu,True,Chevrolet,0.037100
1,15.0,8,350.0,165.0,3693,11.5,1970,1,buick skylark 320,True,Buick,0.044679
2,18.0,8,318.0,150.0,3436,11.0,1970,1,plymouth satellite,True,Other,0.043655
3,16.0,8,304.0,150.0,3433,12.0,1970,1,amc rebel sst,True,Other,0.043694
4,17.0,8,302.0,140.0,3449,10.5,1970,1,ford torino,True,Ford,0.040591


## `groupby()` methods for aggregation

To get aggregated information about subsets of your data. Pretty much every time you use a groupby, you have to pair it with an aggregation function.


Some common aggregations:
* `.min()`: returns the minimum value for each column by group  
* `.max()`: returns the maximum value for each column by group  
* `.mean()`: returns the average value for each column by group  
* `.median()`: returns the median value for each column by group  
* `.count()`: returns the count of each column by group

In [27]:
car_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,heavy,brand,hp_per_weight
0,18.0,8,307.0,130.0,3504,12.0,1970,1,chevrolet chevelle malibu,True,Chevrolet,0.037100
1,15.0,8,350.0,165.0,3693,11.5,1970,1,buick skylark 320,True,Buick,0.044679
2,18.0,8,318.0,150.0,3436,11.0,1970,1,plymouth satellite,True,Other,0.043655
3,16.0,8,304.0,150.0,3433,12.0,1970,1,amc rebel sst,True,Other,0.043694
4,17.0,8,302.0,140.0,3449,10.5,1970,1,ford torino,True,Ford,0.040591


In [28]:
car_df.groupby('cylinders').mean()
# this groups the cylinders and then does all the means for every other column

,mpg,displacement,horsepower,weight,acceleration,year,origin,heavy,hp_per_weight
cylinders,,,,,,,,,
3,20.550000,72.500000,99.250000,2398.500000,13.250000,1975.500000,3.000000,0.000000,0.041442
4,29.317734,109.593596,78.281407,2304.546798,16.582266,1977.049261,1.990148,0.083744,0.034046
5,27.366667,145.000000,82.333333,3103.333333,18.633333,1979.000000,2.000000,1.000000,0.026974
6,19.985714,218.142857,101.506024,3198.226190,16.263095,1975.928571,1.190476,0.892857,0.032013
8,14.963107,345.009709,158.300971,4114.718447,12.955340,1973.902913,1.000000,1.000000,0.038599


In [29]:
car_df.groupby('cylinders').mean()[['mpg', 'weight']]
# for just certain columns

,mpg,weight
cylinders,,
3,20.550000,2398.500000
4,29.317734,2304.546798
5,27.366667,3103.333333
6,19.985714,3198.226190
8,14.963107,4114.718447


In [30]:
# with count
car_df.groupby('cylinders').count()[['mpg', 'weight']]


,mpg,weight
cylinders,,
3,4,4
4,203,203
5,3,3
6,84,84
8,103,103


## Combining DataFrames

### `pd.concat()` and `pd.join()`

`pd.concat` stacks DataFrames together while maintaining columns or index.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

In [31]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])


df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'], 
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])

In [32]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [33]:
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [34]:
df3

,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [35]:
pd.concat([df1, df2, df3])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


In [36]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                    index=[2, 3, 6, 7])
df4

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [37]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [38]:
pd.concat([df1, df4], sort=False)
# gets a lot of NaN volumes

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


In [44]:
pd.concat([df1, df4], join='inner', axis=1) 

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


You can supply the `join` argument to `pd.concat`, but Pandas also has a `.join()` method that can be used interchangably in most cases.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join


There are 4 main types of joins: inner, left, right and outer. This is the same kind of join that SQL uses! https://www.w3schools.com/sql/sql_join.asp

- An inner join only keeps the rows that have indexes in both tables.
- Left join keeps all rows in the left table and supplements that information with info from the right table
- Right join does the opposite of a left join
- Outer join returns a combination of all rows

Also, `pd.merge` exists: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge

In [45]:
pd.concat([df1, df4], join='outer', axis=1) 

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


In [47]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [48]:
df4

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [46]:
df1.join(df4, how='left', rsuffix='4') # why do we need the rsuffix???

,A,B,C,D,B4,D4,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


## Missing Data

Definitely read up on the different ways you can go about filling in missing data.

`df.isna()`, `df.fillna()`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

In [49]:
test = pd.concat([df1, df4], join='outer', axis=1).drop(['B', 'D'], axis=1)
test

,A,C,F
0,A0,C0,NaN
1,A1,C1,NaN
2,A2,C2,F2
3,A3,C3,F3
6,NaN,NaN,F6
7,NaN,NaN,F7


In [50]:
test.isna().sum()

A    2
C    2
F    2
dtype: int64

In [51]:
test.fillna(0)

,A,C,F
0,A0,C0,0
1,A1,C1,0
2,A2,C2,F2
3,A3,C3,F3
6,0,0,F6
7,0,0,F7


In [52]:
# filling NaNs per column
na_values_per_column = {'A': 3, 'C': 2, 'F': 4}
test.fillna(value=na_values_per_column)

,A,C,F
0,A0,C0,4
1,A1,C1,4
2,A2,C2,F2
3,A3,C3,F3
6,3,2,F6
7,3,2,F7


## For 1x1s:

- Attempt `Project: Data Cleaning - Lab`